In [2]:
# com.vn.dic.e.v.ui (17 chars)
a = 2
b = 7

def decode_blob(blob):
    length = 0

    #  create new bytes with the same length blob
    new_blob = bytearray(len(blob))

    if a > 0:
        length = len(blob) // a
    else:
        length = len(blob) // 2

    for i in range(0, length * 2, 2):
        if (i > (b * 2 if b > 0 else 14) + length):
            new_blob[i] = blob[i]
            new_blob[i + 1] = blob[i + 1]
        else:
            new_blob[i] = blob[i + 1]
            new_blob[i + 1] = blob[i]

    if (len(blob) % 2 == 1):
        new_blob[len(blob) - 1] = blob[len(blob) - 1]
    
    return new_blob

# Write the new bytes to the file
# with open(f'{b}.bin', 'wb') as f:
#     f.write(new_blob)

In [5]:
import sqlite3
import gzip

conn = sqlite3.connect('av_all_v3.db')

cursor = conn.cursor()

# TEXT: 125189
# BLOB: 44032
text = 0
blob = 0
for row in cursor.execute('SELECT word, av FROM av'):
    word, av = row
    # Check if av is TEXT or BLOB
    if av[:3] == b'\t\t\t':
        gzip_data = decode_blob(av[3:])
        decompressed = gzip.decompress(gzip_data)
        conn.execute('UPDATE av SET av = ? WHERE word = ?', (decompressed, word))
        conn.commit()
        print(str(blob) + f' Updated {word}')
        blob += 1
    else:
        text += 1

print(f'TEXT: {text}')
print(f'BLOB: {blob}')


TEXT: 169221
BLOB: 0
